# Coarse-grained Simulations
In this notebook, we run coarse-grained simulations of mono-, di-, and tripeptides translocating across a lipid bilayer using the free software __[Faunus](https://mlund.github.io/neofaunus/)__.

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np, pandas as pd
import os, sys, json, shutil
from math import sqrt, pi
import glob
import mdtraj as md
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

/Users/giulio/Repos/SI-peptidebilayer


### Compiling Faunus on a Computer Cluster

In [ ]:
%%bash -s "$workdir"
source activate neomem
rm -rf neofaunus
cd $1
if [ ! -d "neofaunus" ]; then
git clone https://github.com/gitesei/neofaunus.git
cd neofaunus
else
cd neofaunus
rm -rf CMakeCache.txt
fi
module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0
which cmake
cmake --version
cmake . -DENABLE_MPI=ON -DENABLE_OPENMP=OFF -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_CXX_COMPILER=/sw/easybuild/software/Compiler/GCC/7.2.0-2.29/OpenMPI/3.0.0/bin/mpic++ \
    -DCMAKE_C_COMPILER=/sw/easybuild/software/Compiler/GCC/7.2.0-2.29/OpenMPI/3.0.0/bin/mpicc \
    -DPYTHON_EXECUTABLE=/lunarc/nobackup/users/gtesei00/miniconda/envs/neomem/bin/python \
    -DPYTHON_LIBRARY=/lunarc/nobackup/users/gtesei00/miniconda/envs/neomem/lib \
    -DPYTHON_INCLUDE_DIR=/lunarc/nobackup/users/gtesei00/miniconda/envs/neomem/include/python3.6m 
make -j4 faunus
make tests
./tests -d
cd $1

### Launching Faunus Simulations on a Computer Cluster Using SLURM as Resource Manager

In [364]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cp -p mpi*.state $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.7, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"aminoacid": {"atoms": ["AA"], "atomic": True} },
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"aminoacid": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"transrot": {"molecule": "aminoacid", "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e2} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 100000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 4
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
sampl = 10

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)
                    
                    if q == 0:
                        dirname = dirname + '_' + str(aa) 
                
                    #if (pmfs.loc[ah,at,aa,R]['Emax0']>1):
                    #!mkdir $dirname
                    #    print(pmfs.loc[ah,at,aa,R]['Emax0'])
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname

                    %cd $dirname
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp mpi0.penalty.dat $name
                    pmf = []
                    for i in range(20):
                        p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                        pmf.append(p)
                    avg = np.mean(pmf,axis=0)
                    avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    avg[0:58] = avg[-1:58:-1]
                    avg[0] = avg[1]; avg[-1] = avg[-2]
                    avg -= avg[0]
                    np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5
Submitted batch job 1011257
/lunarc/nobackup/users/gtesei00/neomem


### Launching Faunus Simulations of Membrane Translocation of a CG Tripeptide

In [356]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p trimer.xyz bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cp -p mpi*.state $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.7, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"trimer": {"structure": "trimer.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 8*R} } ] } },        
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"trimer": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"moltransrot": {"molecule": "trimer", "dp": 2, "dprot": 0.7, "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "trimer", "repeat": 1} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e4} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 200000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 8
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
sampl = 10

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)+'_trimer'
                    
                    if q == 0:
                        dirname = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_trimer' 
                
                    !mkdir $dirname
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname
                    !cp trimer.xyz $dirname
                    %cd $dirname

                    #pmf = []
                    #for i in range(20):
                    #    p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                    #    pmf.append(p)
                    #np.array(pmf).reshape((int(np.array(pmf).size/2.),2))
                    #avg = np.mean(pmf,axis=0)
                    #avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    #avg[0:58] = avg[-1:58:-1]
                    #avg[0] = avg[1]; avg[-1] = avg[-2]
                    #avg -= avg[0]
                    #np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
mkdir: cannot create directory ‘1_44_-44_2.5_trimer’: File exists
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5_trimer
Submitted batch job 1008604
/lunarc/nobackup/users/gtesei00/neomem


### Launching Faunus Simulations of Membrane Translocation of a CG Titratable Tripeptide

In [376]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p trimer.xyz bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cp -p mpi*.state $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.6, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"trimer": {"structure": "trimer.xyz", "bondlist": [
                {"yukawa": {"index": [0,1], "epsr": 80, "debyelength": 7.86} },
                {"yukawa": {"index": [1,2], "epsr": 80, "debyelength": 7.86} },
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 8*R} } ] } },        
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"trimer": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"moltransrot": {"molecule": "trimer", "dp": 2, "dprot": 0.7, "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "trimer", "repeat": 1} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"swapcharge": {"molecule": "trimer", "repeat": 1, "pH": 7, "pKa": pKa} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e5} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 200000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 8
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
sampl = 10
# samplings
pKa = 12.1

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname0 = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)
                    
                    dirname = dirname0 + '_' + str(aa) + '_trimer_' + str(pKa) 
                
                    #!mkdir $dirname
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname
                    !cp trimer.xyz $dirname
                    %cd $dirname

                    pmf = []
                    for i in range(20):
                        p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                        pmf.append(p)
                    #np.array(pmf).reshape((int(np.array(pmf).size/2.),2))
                    avg = np.mean(pmf,axis=0)
                    avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    avg[0:58] = avg[-1:58:-1]
                    avg[0] = avg[1]; avg[-1] = avg[-2]
                    avg -= avg[0]
                    np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5_2_trimer_12.1
Submitted batch job 1019812
/lunarc/nobackup/users/gtesei00/neomem


### Launching Faunus Simulations of Membrane Translocation of a CG Dipeptide

In [379]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p dimer.xyz bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cp -p mpi*.state $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.7, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"dimer": {"structure": "dimer.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} } ] } },
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"dimer": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"moltransrot": {"molecule": "dimer", "dp": 2, "dprot": 0.7, "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "dimer", "repeat": 1} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e4} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 200000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 8
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
sampl = 10

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)+'_dimer'
                    
                    if q == 0:
                        dirname = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_dimer' 
                
                    #!mkdir $dirname
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname
                    !cp dimer.xyz $dirname
                    %cd $dirname

                    pmf = []
                    for i in range(20):
                        p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                        pmf.append(p)
                    #np.array(pmf).reshape((int(np.array(pmf).size/2.),2))
                    avg = np.mean(pmf,axis=0)
                    avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    avg[0:58] = avg[-1:58:-1]
                    avg[0] = avg[1]; avg[-1] = avg[-2]
                    avg -= avg[0]
                    np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5_dimer
Submitted batch job 1027434
/lunarc/nobackup/users/gtesei00/neomem


### Launching Faunus Simulations of Membrane Translocation of a CG Titratable Dipeptide

In [377]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p dimer.xyz bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cp -p mpi*.state $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.6, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"dimer": {"structure": "dimer.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 3*R, "sigma": 2*R, "eps": eps} },
                {"yukawa": {"index": [0,1], "epsr": 80, "debyelength": 7.86} } ] } },
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"dimer": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"moltransrot": {"molecule": "dimer", "dp": 2, "dprot": 0.7, "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "dimer", "repeat": 1} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"swapcharge": {"molecule": "dimer", "repeat": 1, "pH": 7, "pKa": pKa} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e5} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 200000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 8
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
pKa = 12.1

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname0 = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)
                    
                    dirname = dirname0 + '_' + str(aa) + '_dimer_' + str(pKa)
                
                    #!mkdir $dirname
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname
                    !cp dimer.xyz $dirname
                    %cd $dirname

                    pmf = []
                    for i in range(20):
                        p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                        pmf.append(p)
                    #np.array(pmf).reshape((int(np.array(pmf).size/2.),2))
                    avg = np.mean(pmf,axis=0)
                    avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    avg[0:58] = avg[-1:58:-1]
                    avg[0] = avg[1]; avg[-1] = avg[-2]
                    avg -= avg[0]
                    np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5_2_dimer_12.1
Submitted batch job 1019813
/lunarc/nobackup/users/gtesei00/neomem


### Launching Faunus Simulations of Membrane Translocation of a CG Titratable Monopeptide

In [350]:
%cd $workdir

def mkinput():
    submission_script = """#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -p snic
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --ntasks-per-node=%d
#SBATCH -A snic2018-3-134
#
# job time, change for what your job requires
#SBATCH -t %d:0:0
#
# job name
#SBATCH -J %s
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

module load GCC/7.2.0-2.29
module load OpenMPI/3.0.0

source activate neomem

srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p ../neofaunus/faunus $SNIC_TMP
srun -n $SLURM_NNODES -N $SLURM_NNODES cp -p bilayer.xyz lipid.xyz in.json $SNIC_TMP
cp -p mpi0.penalty.dat $SNIC_TMP
cd $SNIC_TMP
head mpi0.penalty.dat
for ((i=0; i<20; i++))
do
cp -p mpi0.penalty.dat mpi$((i)).penalty.dat 
done
which mpirun
mpirun -bind-to core faunus --nopfx --input in.json
cp -p mpi*.* $SLURM_SUBMIT_DIR
    """
    js = { 
        "temperature": T,
        "random": {"seed": "hardware"},
        "mcloop": {"macro": micro, "micro": macro},
        "geometry": {"length": [L,L,L*2] },
        "energy": [ 
            {"bonded": { } }, {"isobaric": { "P/atm": 0 } },
            {"nonbonded_desernoAA": { 
                "cutoff_g2g": 48, 
                "wca": {"mixing": "LB"},
                "cos2": {"rc": 2**(1/6.)*sigma, "eps": eps, "wc": 16}, 
                "polar": {"epsr":80} } }, 
            {"penalty": { "file": "penalty.dat", "f0": f0, "scale": 0.7, "quiet": False, "update": 1e4, 
                "nodrift":True, "samplings":sampl,
                         "coords": [ {"cmcm": {"range": prange, "type": ['AA','TL'], "index": [],
                                        "dir": [0,0,1], "resolution": 1} } ] } } ],   
        "atomlist": [
            {"TL": {"dp": 2, "sigma": sigma, "eps": eps, "alphax":at} },
            {"HD": {"dp": 4, "sigma": sigma*.95, "eps": eps, "alphax":ah} },
            {"AA": {"dp": 6, "q": q, "sigma":2*R, "eps": eps, "alphax":aa} } ],
        "moleculelist": [
            {"aminoacid": {"atoms": ["AA"], "atomic": True} },
            {"lipid": {"structure": "lipid.xyz", "bondlist": [
                {"fene": {"index": [0,1], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"fene": {"index": [1,2], "k": 30*eps/sigma/sigma, "rmax": 1.5*sigma, "sigma": sigma, "eps": eps} },
                {"harmonic": {"index": [0,2], "k": 10*eps/sigma/sigma, "req": 4*sigma} } ] } } ],
        "insertmolecules": [
            {"lipid": {"N": N, "positions": "bilayer.xyz"} },
            {"aminoacid": {"N": 1} } ],
        "moves": [
            {"moltransrot": {"molecule": "lipid", "dp": 1, "dprot": 0.5, "repeat": N/2} },
            {"transrot": {"molecule": "aminoacid", "repeat": 1, "dir": [0,0,1]} },
            {"transrot": {"molecule": "lipid", "repeat": N/2} },
            {"swapcharge": {"molecule": "aminoacid", "repeat": 1, "pH": 7, "pKa": pKa} },
            {"volume": {"dV": 0.04, "method": "isochoric", "repeat": 1} } ],
        "analysis": [
            {"xtcfile": {"file": "traj.xtc", "nstep": 1e5} },
            {"systemenergy": {"file": "energy.dat", "nstep": 1e5} },
            {"savestate": {"file": "confout.gro"} },
            {"savestate": {"file": "state"} },
            {"savestate": {"file": "confout.pqr"} } ] } 
    with open('in.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))    
    submission_script = submission_script % (cpus,hours,dirname)
    file_handle = open('submission.sh', 'w')
    file_handle.write(submission_script)
    file_handle.close()
    
# number of lipids
N = 162 
# Side length of the box
L = 90
# number of macro MC sweeps
macro = 10
# number of micro MC sweeps
micro = 200000
# penalty range
prange = [-58,58] 
# temperature
T = 293
# epsilon
eps = 8.3145*T*1e-3/1.1
# sigma
sigma = 9
# wall time
hours = 8
# number of cpus
cpus = 20
# f0
f0 = 1e-4
# samplings
sampl = 10
# pKa
pKa = 12.1

for q in [1]:
    for aa in [2]:
        for ah in [44]:
            for at in [-44]:
                for R in [2.5]:
                    dirname0 = str(q)+'_'+str(ah)+'_'+str(at)+'_'+str(R)
                    
                    dirname = dirname0 + '_' + str(aa) + '_' + str(pKa)
                
                    #if (pmfs.loc[ah,at,aa,R]['Emax0']>1):
                    !mkdir $dirname
                    #    print(pmfs.loc[ah,at,aa,R]['Emax0'])
                    !cp bilayer.xyz $dirname
                    !cp lipid.xyz $dirname

                    %cd $dirname
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../$dirname0/$name $name
                    #pmf = []
                    #for i in range(20):
                    #    p = np.loadtxt('mpi'+str(i)+'.penalty.dat')
                    #    pmf.append(p)
                    #avg = np.mean(pmf,axis=0)
                    #avg[58:-1] = .5*( avg[58:-1] + avg[58:0:-1] )
                    #avg[0:58] = avg[-1:58:-1]
                    #avg[0] = avg[1]; avg[-1] = avg[-2]
                    #avg -= avg[0]
                    #np.savetxt('mpi0.penalty.dat',avg,header=str(f0)+' '+str(sampl))

                    #!rm *penalty*
                    #for i in range(20):
                    #    name = 'mpi'+str(i)+'.penalty.dat'
                    #    !cp ../mpi0.penalty.dat $name
                    mkinput()
                    if shutil.which('sbatch') is not None:
                        !sbatch submission.sh  # run on slurm cluster
                    else:
                        !mpirun -np 2 --stdin all ../neofaunus/faunus < in.json

                    %cd $workdir

/lunarc/nobackup/users/gtesei00/neomem
/lunarc/nobackup/users/gtesei00/neomem/1_44_-44_2.5_2_12.1
Submitted batch job 1008598
/lunarc/nobackup/users/gtesei00/neomem


### Pickle File for All PMFs

In [174]:
if os.path.isfile('pmfs.p'):
    pmfs = pd.read_pickle('pmfs.p')
else:
    data = []
    for ah in [25,30,35]:
        for at in [-30,-35]:
            for R in [1.5,2,2.5,3]:
                dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)
                if os.path.isfile(dirname+'/mpi0.penalty.dat'):
                    y1, Ey1 = averagePMFs(dirname)
                    for aa in [1,2]:
                        dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)
                        if os.path.isfile(dirname+'/mpi0.penalty.dat'):
                            y0, Ey0 = averagePMFs(dirname)
                            x = np.linspace(0,56,56+1)
                            data.append({'aa':aa,'ah':ah,'at':at,'R':R,
                                         'min0':y0.min(),
                                         'max0':y0.max(),
                                         'Emin0':Ey0[y0==y0.min()][0],
                                         'Emax0':Ey0[y0==y0.max()][0],
                                         'xmin0':x[y0==y0.min()],
                                         'x0':x,'y0':y0,'Ey0':Ey0,
                                         'min1':y1.min(),
                                         'max1':y1.max(),
                                         'Emin1':Ey1[y1==y1.min()][0],
                                         'Emax1':Ey1[y1==y1.max()][0],
                                         'xmin1':x[y1==y1.min()],
                                         'x1':x,'y1':y1,'Ey1':Ey1})
    for R in [1,1.5,2,2.5,3,3.5,4]:
        dirname = '0_0_0_'+str(R)+'_0'
        y, Ey = averagePMFs(dirname)
        x = np.linspace(0,56,56+1)
        data.append({'aa':0,'ah':0,'at':0,'R':R,
                     'min0':y.min(),
                     'max0':y.max(),
                     'Emin0':Ey[y==y.min()][0],
                     'Emax0':Ey[y==y.max()][0],
                     'xmin0':x[y==y.min()],
                     'x0':x,'y0':y,'Ey0':Ey,
                     'min1':y.min(),
                     'max1':y.max(),
                     'Emin1':Ey[y==y.min()][0],
                     'Emax1':Ey[y==y.max()][0],
                     'xmin1':x[y==y.min()],
                     'x1':x,'y1':y,'Ey1':Ey})
    pmfs = pd.DataFrame(data)
    pmfs = pmfs.set_index(['ah','at','aa','R'])
    pmfs.to_pickle('pmfs.p')

### Pickle File for Titratable Amino Acids

In [368]:
def averagePMFs(dirname):
    pmf = []
    cnt=0
    for filename in glob.glob(dirname+'/mpi*.penalty.dat'):
        p = np.loadtxt(filename)
        pmf.append(-p)
        cnt += 1
    if (cnt!=20): print('less than 20 PMFs!!!')
    avg = np.mean(pmf,axis=0)
    std = np.std(pmf,axis=0)
    x = np.linspace(-58,58,58*2+1)
    u = avg[58:-2]; v = avg[58:1:-1]
    u = u - u[-1]
    v = v - v[-1]
    y = .5*(u + v)
    Ey = np.abs(.5*(u - v))
    return y,Ey

data = []
aa=2; ah=25; at=-35; R=2
dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y1, Ey1 = averagePMFs(dirname)
dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y0, Ey0 = averagePMFs(dirname)
x = np.linspace(0,56,56+1)
for pKa in [6.04,10.67]:
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+str(pKa)
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        yt, Eyt = averagePMFs(dirname)
    data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                 'min0':y0.min(),
                 'max0':y0.max(),
                 'Emin0':Ey0[y0==y0.min()][0],
                 'Emax0':Ey0[y0==y0.max()][0],
                 'xmin0':x[y0==y0.min()],
                 'x0':x,'y0':y0,'Ey0':Ey0,
                 'min1':y1.min(),
                 'max1':y1.max(),
                 'Emin1':Ey1[y1==y1.min()][0],
                 'Emax1':Ey1[y1==y1.max()][0],
                 'xmin1':x[y1==y1.min()],
                 'x1':x,'y1':y1,'Ey1':Ey1,
                 'min0/1':yt.min(),
                 'max0/1':yt.max(),
                 'Emin0/1':Eyt[yt==yt.min()][0],
                 'Emax0/1':Eyt[yt==yt.max()][0],
                 'xmin0/1':x[yt==yt.min()],
                 'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
aa=3; ah=35; at=-35; R=2.5
dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y1, Ey1 = averagePMFs(dirname)
dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y0, Ey0 = averagePMFs(dirname)
x = np.linspace(0,56,56+1)
for pKa in [12.1]:
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+str(pKa)
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        yt, Eyt = averagePMFs(dirname)
    data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                 'min0':y0.min(),
                 'max0':y0.max(),
                 'Emin0':Ey0[y0==y0.min()][0],
                 'Emax0':Ey0[y0==y0.max()][0],
                 'xmin0':x[y0==y0.min()],
                 'x0':x,'y0':y0,'Ey0':Ey0,
                 'min1':y1.min(),
                 'max1':y1.max(),
                 'Emin1':Ey1[y1==y1.min()][0],
                 'Emax1':Ey1[y1==y1.max()][0],
                 'xmin1':x[y1==y1.min()],
                 'x1':x,'y1':y1,'Ey1':Ey1,
                 'min0/1':yt.min(),
                 'max0/1':yt.max(),
                 'Emin0/1':Eyt[yt==yt.min()][0],
                 'Emax0/1':Eyt[yt==yt.max()][0],
                 'xmin0/1':x[yt==yt.min()],
                 'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
aa=2; ah=44; at=-44; R=2.5
dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y1, Ey1 = averagePMFs(dirname)
dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)
if os.path.isfile(dirname+'/mpi0.penalty.dat'):
    y0, Ey0 = averagePMFs(dirname)
x = np.linspace(0,56,56+1)
for pKa in [12.1]:
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+str(pKa)
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        yt, Eyt = averagePMFs(dirname)
    data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                 'min0':y0.min(),
                 'max0':y0.max(),
                 'Emin0':Ey0[y0==y0.min()][0],
                 'Emax0':Ey0[y0==y0.max()][0],
                 'xmin0':x[y0==y0.min()],
                 'x0':x,'y0':y0,'Ey0':Ey0,
                 'min1':y1.min(),
                 'max1':y1.max(),
                 'Emin1':Ey1[y1==y1.min()][0],
                 'Emax1':Ey1[y1==y1.max()][0],
                 'xmin1':x[y1==y1.min()],
                 'x1':x,'y1':y1,'Ey1':Ey1,
                 'min0/1':yt.min(),
                 'max0/1':yt.max(),
                 'Emin0/1':Eyt[yt==yt.min()][0],
                 'Emax0/1':Eyt[yt==yt.max()][0],
                 'xmin0/1':x[yt==yt.min()],
                 'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
titr = pd.DataFrame(data)
titr = titr.set_index(['ah','at','aa','R','pKa'])
titr.to_pickle('titr.p')

### Pickle File for Dimers and Trimers

In [378]:
def averagePMFs(dirname):
    pmf = []
    cnt=0
    for filename in glob.glob(dirname+'/mpi*.penalty.dat'):
        p = np.loadtxt(filename)
        pmf.append(-p)
        cnt += 1
    if (cnt!=20): print('less than 20 PMFs!!!')
    avg = np.mean(pmf,axis=0)
    std = np.std(pmf,axis=0)
    x = np.linspace(-58,58,58*2+1)
    u = avg[58:-2]; v = avg[58:1:-1]
    u = u - u[-1]
    v = v - v[-1]
    y = .5*(u + v)
    Ey = np.abs(.5*(u - v))
    return y,Ey

for pep in ['dimer','trimer']:
    data = []
    aa=2; ah=25; at=-35; R=2
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y1, Ey1 = averagePMFs(dirname)
    dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y0, Ey0 = averagePMFs(dirname)
    x = np.linspace(0,56,56+1)
    for pKa in [6.04,10.67]:
        dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep+'_'+str(pKa)
        if os.path.isfile(dirname+'/mpi0.penalty.dat'):
            yt, Eyt = averagePMFs(dirname)
        data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                     'min0':y0.min(),
                     'max0':y0.max(),
                     'Emin0':Ey0[y0==y0.min()][0],
                     'Emax0':Ey0[y0==y0.max()][0],
                     'xmin0':x[y0==y0.min()],
                     'x0':x,'y0':y0,'Ey0':Ey0,
                     'min1':y1.min(),
                     'max1':y1.max(),
                     'Emin1':Ey1[y1==y1.min()][0],
                     'Emax1':Ey1[y1==y1.max()][0],
                     'xmin1':x[y1==y1.min()],
                     'x1':x,'y1':y1,'Ey1':Ey1,
                     'min0/1':yt.min(),
                     'max0/1':yt.max(),
                     'Emin0/1':Eyt[yt==yt.min()][0],
                     'Emax0/1':Eyt[yt==yt.max()][0],
                     'xmin0/1':x[yt==yt.min()],
                     'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
    aa=3; ah=35; at=-35; R=2.5
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y1, Ey1 = averagePMFs(dirname)
    dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y0, Ey0 = averagePMFs(dirname)
    x = np.linspace(0,56,56+1)
    for pKa in [12.1]:
        dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep+'_'+str(pKa)
        if os.path.isfile(dirname+'/mpi0.penalty.dat'):
            yt, Eyt = averagePMFs(dirname)
        data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                     'min0':y0.min(),
                     'max0':y0.max(),
                     'Emin0':Ey0[y0==y0.min()][0],
                     'Emax0':Ey0[y0==y0.max()][0],
                     'xmin0':x[y0==y0.min()],
                     'x0':x,'y0':y0,'Ey0':Ey0,
                     'min1':y1.min(),
                     'max1':y1.max(),
                     'Emin1':Ey1[y1==y1.min()][0],
                     'Emax1':Ey1[y1==y1.max()][0],
                     'xmin1':x[y1==y1.min()],
                     'x1':x,'y1':y1,'Ey1':Ey1,
                     'min0/1':yt.min(),
                     'max0/1':yt.max(),
                     'Emin0/1':Eyt[yt==yt.min()][0],
                     'Emax0/1':Eyt[yt==yt.max()][0],
                     'xmin0/1':x[yt==yt.min()],
                     'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
    aa=2; ah=44; at=-44; R=2.5
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y1, Ey1 = averagePMFs(dirname)
    dirname = '0_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        y0, Ey0 = averagePMFs(dirname)
    x = np.linspace(0,56,56+1)
    for pKa in [12.1]:
        dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+str(aa)+'_'+pep+'_'+str(pKa)
        if os.path.isfile(dirname+'/mpi0.penalty.dat'):
            yt, Eyt = averagePMFs(dirname)
        data.append({'aa':aa,'ah':ah,'at':at,'R':R,'pKa':pKa,
                     'min0':y0.min(),
                     'max0':y0.max(),
                     'Emin0':Ey0[y0==y0.min()][0],
                     'Emax0':Ey0[y0==y0.max()][0],
                     'xmin0':x[y0==y0.min()],
                     'x0':x,'y0':y0,'Ey0':Ey0,
                     'min1':y1.min(),
                     'max1':y1.max(),
                     'Emin1':Ey1[y1==y1.min()][0],
                     'Emax1':Ey1[y1==y1.max()][0],
                     'xmin1':x[y1==y1.min()],
                     'x1':x,'y1':y1,'Ey1':Ey1,
                     'min0/1':yt.min(),
                     'max0/1':yt.max(),
                     'Emin0/1':Eyt[yt==yt.min()][0],
                     'Emax0/1':Eyt[yt==yt.max()][0],
                     'xmin0/1':x[yt==yt.min()],
                     'x0/1':x,'y0/1':yt,'Ey0/1':Eyt})
    titr = pd.DataFrame(data)
    titr = titr.set_index(['ah','at','aa','R','pKa'])
    titr.to_pickle(pep+'.p')

### Pickle File for PMFs Reproducing Results by __[Hu et al.](https://pubs.acs.org/doi/10.1021/jp504853t)__

In [333]:
def averagePMFs(dirname):
    pmf = []
    cnt=0
    for filename in glob.glob(dirname+'/mpi*.penalty.dat'):
        p = np.loadtxt(filename)
        pmf.append(-p)
        cnt += 1
    if (cnt!=20): print('less than 20 PMFs!!!')
    avg = np.mean(pmf,axis=0)
    std = np.std(pmf,axis=0)
    x = np.linspace(-58,58,58*2+1)
    u = avg[58:-2]; v = avg[58:1:-1]
    u = u - u[-1]
    v = v - v[-1]
    y = .5*(u + v)
    Ey = np.abs(.5*(u - v))
    return y,Ey

data = []
ah=11; at=-3; R=2.5
x = np.linspace(0,56,56+1)
for pep in ['monomer','dimer','trimer']:
    dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)+'_'+pep
    if pep=='monomer':
        dirname = '1_'+str(ah)+'_'+str(at)+'_'+str(R)
    if os.path.isfile(dirname+'/mpi0.penalty.dat'):
        print(dirname)
        y1, Ey1 = averagePMFs(dirname)
    data.append({'ah':ah,'at':at,'R':R,'pep':pep,
                 'min1':y1.min(),
                 'max1':y1.max(),
                 'Emin1':Ey1[y1==y1.min()][0],
                 'Emax1':Ey1[y1==y1.max()][0],
                 'xmin1':x[y1==y1.min()],
                 'x1':x,'y1':y1,'Ey1':Ey1})
hu = pd.DataFrame(data)
hu = hu.set_index(['ah','at','R','pep'])
hu.to_pickle('hu.p')

1_11_-3_2.5
1_11_-3_2.5_dimer
1_11_-3_2.5_trimer
